In [1]:
import requests
from requests import get
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np


In [18]:
#geopy installation

!pip install geopy
import geopy 

from geopy.geocoders import Nominatim
import time
from pprint import pprint

app = Nominatim(user_agent="cphDorms")

In [3]:
#Installing shortuuid to make dorms identifiable (Some Dorm Names Suck!)

!pip install shortuuid

import shortuuid    
print(shortuuid.ShortUUID().random(length=6)) #Example Code 

3Zh66k


In [4]:
url = "https://www.kollegierneskontor.dk/default.aspx?func=kkikportal.kollegiumlist&mid=40&topmenuid=34&lang=GB"
results = requests.get(url)

In [5]:
soup = BeautifulSoup(results.text, "html.parser")


In [16]:
#Longitude and latitude

def coordinates(dorm_address):
    def get_location_by_address(address):
        time.sleep(1.5)
        try:
             return app.geocode(address).raw
        except:
            print("Couldn'd find coordinates of " + address)
            return {"lat":"NaN", "lon":"NaN"}
    address = dorm_address
    location = get_location_by_address(address)
    latitude = location["lat"]
    longitude = location["lon"]
    return [latitude, longitude]

#print(f"{latitude}, {longitude}")
# print all returned data
#pprint(location)

print(coordinates("Dalslandsgade 8, Copenhagen S"))

['55.667422', '12.60431']


In [19]:

kkik = []

kollegium = soup.find_all('div', class_='kollegium row')


for container in kollegium:
    print("Processing Dorm")
    dorm_name = container.h3.a.text
    dorm_address = container.div.find('div', class_="address").text.strip().replace("\r\n               ", "").replace("\xa0", "")

    body = container.find('div', class_="body col-xs-12")
    rows = body.find_all("div", class_= "row")
    rows.pop(0)
    rows.pop(0)
    rows.pop(0)

    rooms_export = []

    for room in rows:
        room_details = room.find_all("div")

        roomtype = room_details[0].get_text().strip()
        size = room_details[1].get_text().strip()
        price = room_details[3].get_text().strip()

        rooms_export.append({
            "roomType":roomtype,
            "size":size,
            "price":price
        })

    kkik.append({
        "name":dorm_name,
        "address":dorm_address,
        "coordinates":coordinates(dorm_address),
        "rooms":rooms_export,
        "UUID":shortuuid.ShortUUID().random(length=6)
    })








Processing Dorm
Processing Dorm
Processing Dorm
Processing Dorm
Processing Dorm
Couldn'd find coordinates of Emdrupvej 54 A, 2400 København NV
Processing Dorm
Processing Dorm
Processing Dorm
Processing Dorm
Processing Dorm
Couldn'd find coordinates of Holmbladsgade 70 C, 2300 København S
Processing Dorm
Processing Dorm
Processing Dorm
Couldn'd find coordinates of Kagsåkollegiet 2 , 2860 Søborg
Processing Dorm
Processing Dorm
Processing Dorm
Processing Dorm
Processing Dorm
Processing Dorm
Processing Dorm
Processing Dorm
Couldn'd find coordinates of Robert Jacobsensvej 61 , 2300 København S
Processing Dorm
Processing Dorm
Processing Dorm
Couldn'd find coordinates of Rektorparken 12 , 2450 København SV
Processing Dorm
Couldn'd find coordinates of Stærevej 66 , 2400 København NV
Processing Dorm
Processing Dorm
Processing Dorm


In [42]:
import json

with open('json_kkik.json', 'w') as f:
    json.dump(kkik, f)


3Y9pHV
